In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn import preprocessing

In [ ]:
data = pd.read_csv("/content/drive/My Drive/5001/kaggle/train.csv")
Y = data['y']
X = data.iloc[:,:86]

In [ ]:
std = np.std(Y, axis=0)
mean = np.mean(Y, axis=0)
scaler = StandardScaler().fit(X.iloc[:,:45])
X.iloc[:,:45] = scaler.transform(X.iloc[:,:45])
for i in range(len(X)):
  for j in range(45,86):
    X.iloc[i,j] = (X.iloc[i,j]-mean)/std
Y = (Y-mean)/std
# if normalization

In [ ]:
X

,day,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,time0,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12,time13,time14,time15,time16,time17,time18,time19,time20,time21,time22,time23,weekday1,weekday2,weekday3,...,t14,t15,t16,t17,t18,t19,t20,t21,t22,t26,t27,t28,t29,t30,t31,t32,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,t71,t72,t73,t74,t75,t76,t77,t78,t79,t80
0,-0.910968,3.727271,-0.291269,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.208502,4.789147,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.407943,-0.407555,-0.407555,...,0.855248,0.619242,1.012831,1.038366,0.863349,0.925591,0.956673,1.158165,1.232871,0.988697,1.058289,1.218275,1.190819,1.224226,1.204325,1.139619,1.139619,1.113770,1.194251,1.262217,1.108221,1.239502,0.552027,-0.760610,-0.741147,-0.490006,-1.411098,-0.944592,-1.113685,-0.431415,-1.092400,-1.171940,-1.543883,-0.736551,-0.076204,0.629666,0.553438,0.188286,0.054785,0.909029
1,-0.910968,3.727271,-0.291269,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.208502,-0.208805,4.796123,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.407943,-0.407555,-0.407555,...,0.619242,1.012831,1.038366,0.863349,0.925591,0.956673,1.158165,1.232871,0.685147,0.855248,1.045472,1.012360,1.168676,1.061707,1.212772,1.113770,1.113770,1.194251,1.262217,1.108221,1.239502,0.552027,-0.760610,-0.741147,-0.490006,-1.411098,-0.944592,-1.113685,-0.431415,-1.092400,-1.171940,-1.543883,-0.736551,-0.076204,0.629666,0.553438,0.188286,0.054785,0.909029,0.999766
2,-0.910968,3.727271,-0.291269,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.208502,-0.208805,-0.208502,4.796123,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.407943,-0.407555,-0.407555,...,1.012831,1.038366,0.863349,0.925591,0.956673,1.158165,1.232871,0.685147,0.740457,0.619242,1.057648,1.189236,1.222310,1.228496,1.182115,1.194251,1.194251,1.262217,1.108221,1.239502,0.552027,-0.760610,-0.741147,-0.490006,-1.411098,-0.944592,-1.113685,-0.431415,-1.092400,-1.171940,-1.543883,-0.736551,-0.076204,0.629666,0.553438,0.188286,0.054785,0.909029,0.999766,0.954636
3,-0.910968,3.727271,-0.291269,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.208502,-0.208805,-0.208502,-0.208502,4.796123,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.407943,-0.407555,-0.407555,...,1.038366,0.863349,0.925591,0.956673,1.158165,1.232871,0.685147,0.740457,1.121652,1.012831,1.054339,1.064219,1.165262,1.132685,1.049019,1.262217,1.262217,1.108221,1.239502,0.552027,-0.760610,-0.741147,-0.490006,-1.411098,-0.944592,-1.113685,-0.431415,-1.092400,-1.171940,-1.543883,-0.736551,-0.076204,0.629666,0.553438,0.188286,0.054785,0.909029,0.999766,0.954636,0.985282
4,-0.910968,3.727271,-0.291269,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.307878,-0.30241,-0.307878,-0.30241,-0.307878,-0.208502,-0.208805,-0.208502,-0.208502,-0.208502,4.796123,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.208502,-0.407943,-0.407555,-0.407555,...,0.863349,0.925591,0.956673,1.158165,1.232871,0.685147,0.740457,1.121652

In [ ]:
param_grid = [{'learning_rate': [0.05], 'max_depth': [20], 'num_leaves': [45], 'n_estimators': [200], 'feature_fraction': [0.9], 'bagging_fraction': [0.7]}]
model = GridSearchCV(lgb.LGBMRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

model.fit(X, Y)
#model = (learning_rate=0.05,max_depth=20,n_estimators=200,num_leaves=40).fit(X, Y)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid=[{'bagging_fraction': [0.7, 0.8],
                          'feature_fraction': [0.7, 0.9],
                          'learning_rate': [0.03, 0.05, 0.07],
 

In [ ]:
print(model.best_score_*std*std)
print(model.best_estimator_)

-11.891806508225065
LGBMRegressor(bagging_fraction=0.7, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=20,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=200, n_jobs=-1, num_leaves=45, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


#Evaluation

In [1]:
test = pd.read_csv("/content/drive/My Drive/5001/kaggle/eval.csv")
y = test['y']
x = test.iloc[:,1:86]

NameError: ignored

In [ ]:
for i in range(len(y)):
  a = model.predict([x.iloc[i,:]])[0]
  if np.isnan(y.iloc[i]):
    y.iloc[i] = a
    for j in range(24):
      if i+j+1<len(y):
        x.iloc[i+j+1,84-j] = a
    for k in range(7):
      if i+(k+1)*24<len(y):
        x.iloc[i+(k+1)*24,60-k] = a
    for p in range(10):
      if i+p+159<len(y):
        x.iloc[i+p+159,53-p] = a
  else:
    y.iloc[i] = a

NameError: ignored

In [ ]:
y.to_csv('/content/drive/My Drive/5001/kaggle/t.csv')